In [ ]:
{
  "cells": [
    {
      "cell_type": "code",
      "source": [
        "# task3\n",
        "from selenium import webdriver\n",
        "from selenium.webdriver.chrome.service import Service as ChromeService\n",
        "from selenium.webdriver.chrome.options import Options\n",
        "from webdriver_manager.chrome import ChromeDriverManager\n",
        "# ... other imports ...\n",
        "\n",
        "def get_colab_driver():\n",
        "    \"\"\"Sets up a headless Chrome driver configured for Google Colab.\"\"\"\n",
        "    chrome_options = Options()\n",
        "    chrome_options.add_argument(\"--headless\")\n",
        "    chrome_options.add_argument(\"--no-sandbox\")\n",
        "    chrome_options.add_argument(\"--disable-dev-shm-usage\")\n",
        "\n",
        "    driver = webdriver.Chrome(\n",
        "        service=ChromeService(ChromeDriverManager().install()),\n",
        "        options=chrome_options\n",
        "    )\n",
        "    return driver\n",
        "\n",
        "class AutomatedLoginTest:\n",
        "    def __init__(self):\n",
        "\n",
        "        self.driver = get_colab_driver()\n",
        "        self.driver.implicitly_wait(10)\n",
        "\n",
        "\n",
        "# ... main() function to execute the tests ..."
      ],
      "metadata": {
        "id": "5M3K9faXegqd"
      },
      "execution_count": 7,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# --- Cell 1: Import Libraries and Load Data ---\n",
        "import pandas as pd\n",
        "import numpy as np\n",
        "from sklearn.datasets import load_breast_cancer\n",
        "from sklearn.model_selection import train_test_split\n",
        "from sklearn.ensemble import RandomForestClassifier\n",
        "from sklearn.metrics import accuracy_score, f1_score, classification_report\n",
        "\n",
        "print(\"Libraries loaded successfully.\")\n",
        "\n",
        "# Load the Breast Cancer dataset (used as proxy for 'GitHub Issues' data)\n",
        "data = load_breast_cancer(as_frame=True)\n",
        "df = data.frame\n",
        "X = df.drop(columns=['target']) # Features (e.g., code metrics, past issue history)\n",
        "y = df['target']             # Original target (0=Benign, 1=Malignant)\n",
        "\n",
        "print(f\"Original dataset shape: {df.shape}\")\n",
        "print(f\"Features (X) shape: {X.shape}\")\n",
        "print(f\"Target (y) shape: {y.shape}\")\n",
        "print(\"First 5 rows of data:\")\n",
        "print(df.head())"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "r7CFmdAye5a8",
        "outputId": "a431ed78-3c78-462c-9646-eddb75be1fea"
      },
      "execution_count": 3,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Libraries loaded successfully.\n",
            "Original dataset shape: (569, 31)\n",
            "Features (X) shape: (569, 30)\n",
            "Target (y) shape: (569,)\n",
            "First 5 rows of data:\n",
            "   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \\\n",
            "0        17.99         10.38          122.80     1001.0          0.11840   \n",
            "1        20.57         17.77          132.90     1326.0          0.08474   \n",
            "2        19.69         21.25          130.00     1203.0          0.10960   \n",
            "3        11.42         20.38           77.58      386.1          0.14250   \n",
            "4        20.29         14.34          135.10     1297.0          0.10030   \n",
            "\n",
            "   mean compactness  mean concavity  mean concave points  mean symmetry  \\\n",
            "0           0.27760          0.3001              0.14710         0.2419   \n",
            "1           0.07864          0.0869              0.07017         0.1812   \n",
            "2           0.15990          0.1974              0.12790         0.2069   \n",
            "3           0.28390          0.2414              0.10520         0.2597   \n",
            "4           0.13280          0.1980              0.10430         0.1809   \n",
            "\n",
            "   mean fractal dimension  ...  worst texture  worst perimeter  worst area  \\\n",
            "0                 0.07871  ...          17.33           184.60      2019.0   \n",
            "1                 0.05667  ...          23.41           158.80      1956.0   \n",
            "2                 0.05999  ...          25.53           152.50      1709.0   \n",
            "3                 0.09744  ...          26.50            98.87       567.7   \n",
            "4                 0.05883  ...          16.67           152.20      1575.0   \n",
            "\n",
            "   worst smoothness  worst compactness  worst concavity  worst concave points  \\\n",
            "0            0.1622             0.6656           0.7119                0.2654   \n",
            "1            0.1238             0.1866           0.2416                0.1860   \n",
            "2            0.1444             0.4245           0.4504                0.2430   \n",
            "3            0.2098             0.8663           0.6869                0.2575   \n",
            "4            0.1374             0.2050           0.4000                0.1625   \n",
            "\n",
            "   worst symmetry  worst fractal dimension  target  \n",
            "0          0.4601                  0.11890       0  \n",
            "1          0.2750                  0.08902       0  \n",
            "2          0.3613                  0.08758       0  \n",
            "3          0.6638                  0.17300       0  \n",
            "4          0.2364                  0.07678       0  \n",
            "\n",
            "[5 rows x 31 columns]\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# --- Cell 2: Preprocessing, Feature Engineering (Custom Labeling) ---\n",
        "\n",
        "# 1. Create a \"High Priority\" based on the original malignant target (1)\n",
        "df['Priority'] = df['target'].map({0: 'Low', 1: 'High'})\n",
        "\n",
        "# 2. Introduce a \"Medium Priority\" based on a feature split (e.g., \"mean texture\")\n",
        "# We assume that issues with intermediate complexity (mean texture) are \"Medium\"\n",
        "texture_median = df['mean texture'].median()\n",
        "\n",
        "def assign_priority(row):\n",
        "    # If it's already \"High\" (Malignant), keep it High\n",
        "    if row['Priority'] == 'High':\n",
        "        return 'High'\n",
        "    # If it's \"Low\" (Benign) but has a high 'mean texture' (complex characteristics),\n",
        "    # upgrade it to Medium Priority for resource allocation.\n",
        "    elif row['mean texture'] >= texture_median:\n",
        "        return 'Medium'\n",
        "    else:\n",
        "        return 'Low'\n",
        "\n",
        "# Apply the new logic\n",
        "df['Priority'] = df.apply(assign_priority, axis=1)\n",
        "\n",
        "# Display the distribution of the new custom priority labels\n",
        "priority_counts = df['Priority'].value_counts()\n",
        "print(\"Distribution of new custom 'Priority' labels:\")\n",
        "print(priority_counts)\n",
        "\n",
        "# Define the new features (X_new) and the target (y_new)\n",
        "X_new = df.drop(columns=['target', 'Priority'])\n",
        "y_new = df['Priority']\n",
        "\n",
        "# Split the data into Training and Testing sets (70% Train, 30% Test)\n",
        "X_train, X_test, y_train, y_test = train_test_split(\n",
        "    X_new, y_new, test_size=0.3, random_state=42, stratify=y_new\n",
        ")\n",
        "\n",
        "print(f\"\\nTraining set size: {X_train.shape[0]} samples\")\n",
        "print(f\"Testing set size: {X_test.shape[0]} samples\")"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "PFiHEjCufRgi",
        "outputId": "93d50a79-96a7-4c49-9234-3f9111551505"
      },
      "execution_count": 4,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Distribution of new custom 'Priority' labels:\n",
            "Priority\n",
            "High      357\n",
            "Medium    166\n",
            "Low        46\n",
            "Name: count, dtype: int64\n",
            "\n",
            "Training set size: 398 samples\n",
            "Testing set size: 171 samples\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# --- Cell 3: Model Training (Random Forest) ---\n",
        "\n",
        "# Initialize the Random Forest Classifier\n",
        "\n",
        "model = RandomForestClassifier(n_estimators=100, random_state=42)\n",
        "\n",
        "print(\"Starting Random Forest Model Training...\")\n",
        "\n",
        "# Train the model using the training data\n",
        "model.fit(X_train, y_train)\n",
        "\n",
        "print(\"Model training complete!\")"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "V6ftGGsYf3hB",
        "outputId": "6fcecdcf-4ef2-4c0c-8246-65b5593937a8"
      },
      "execution_count": 8,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Starting Random Forest Model Training...\n",
            "Model training complete!\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# --- Cell 4: Evaluation and Performance Metrics ---\n",
        "\n",
        "# Make predictions on the test set\n",
        "y_pred = model.predict(X_test)\n",
        "\n",
        "# --- Calculate Required Metrics ---\n",
        "\n",
        "# 1. Accuracy\n",
        "accuracy = accuracy_score(y_test, y_pred)\n",
        "print(f\"Accuracy Score: {accuracy:.4f}\")\n",
        "\n",
        "# 2. F1-Score (using 'weighted' for multi-class problem)\n",
        "f1 = f1_score(y_test, y_pred, average='weighted')\n",
        "print(f\"F1-Score (Weighted): {f1:.4f}\")\n",
        "\n",
        "# --- Detailed Classification Report ---\n",
        "\n",
        "# Provides precision, recall, and F1-score for each category (Low, Medium, High)\n",
        "print(\"\\n--- Detailed Classification Report ---\")\n",
        "print(classification_report(y_test, y_pred))\n",
        "\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "zJdQ4BJ_f_lz",
        "outputId": "b62d13c6-5992-4914-dfb9-90f9d4fb365f"
      },
      "execution_count": 9,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Accuracy Score: 0.9649\n",
            "F1-Score (Weighted): 0.9632\n",
            "\n",
            "--- Detailed Classification Report ---\n",
            "              precision    recall  f1-score   support\n",
            "\n",
            "        High       0.96      1.00      0.98       107\n",
            "         Low       1.00      0.71      0.83        14\n",
            "      Medium       0.96      0.96      0.96        50\n",
            "\n",
            "    accuracy                           0.96       171\n",
            "   macro avg       0.97      0.89      0.92       171\n",
            "weighted avg       0.97      0.96      0.96       171\n",
            "\n"
          ]
        }
      ]
    }
  ],
  "metadata": {
    "colab": {
      "name": "Snippets: Importing libraries",
      "provenance": []
    },
    "kernelspec": {
      "display_name": "Python 3",
      "name": "python3"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 0
}